# Train models to predict faiure probability at hourly resolution

## Features :

*Weather*
-
-
-

*Load*
-
-
-

*Chronological*
-
-
-

In [1]:
# Import libraries

# Data processing and manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.cluster import KMeans
import xgboost as xgb

# Other
import importlib
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))
import inferenceModels as im    # your file



# Mutual information
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif



from typing import Iterable
from pathlib import Path



importlib.reload(im)

<module 'inferenceModels' from '/Users/malo/MIT Dropbox/Malo Lahogue/Research/Generator-outage-probability/src/inferenceModels.py'>

## Load data

## Get features and data source

In [2]:
weather_data_file = "../Data/hourly/hourly_weather_by_state.csv"
power_load_file = "../Data/hourly/hourly_load_by_state.csv"
failure_data_file = "../Data/hourly/hourly_failure_dataset_compressed.csv"

weather_data = pd.read_csv(weather_data_file, parse_dates=["datetime"])
power_load_data = pd.read_csv(power_load_file, parse_dates=["UTC time"])

In [3]:
feature_names = list(weather_data.columns) + list(power_load_data.columns) + ['Season', 'Month', 'DayOfWeek', 'DayOfYear', 'Holiday', 'Weekend', 'Technology']
feature_names = list(set(feature_names)-set(['datetime', 'UTC time', 'Datetime_UTC', 'Datetime']))
feature_names = list(set([(name[0].upper() + name[1:]) if isinstance(name, str) and name else name for name in feature_names]))
print(f"{len(feature_names)} features:", feature_names)

39 features: ['Extreme_heat', 'Tmax', 'HDD', 'HDD3d', 'DayOfYear', 'Tmin', 'Wet_bulb_temperature', 'Load', '1d_load_sum', 'Precipitation', 'Weekend', 'CDD3d', '24h_max_load', 'Dew_point_temperature', 'Holiday', 'Wind_chill', 'Precip_1d', 'Month', 'FDD', 'Pressure_3hr_change', 'Tmean', 'Technology', 'Wind_speed', 'Extreme_cold', 'Hourly_load_change', 'FDD3d', '2d_load_sum', 'Season', 'Station_level_pressure', 'State', 'Sea_level_pressure', 'Temperature', 'Relative_humidity', 'CDD', 'Snow_depth', 'Heat_index', 'Precip_3d', 'DayOfWeek', '24h_min_load']


## Test and count per technology

In [7]:
importlib.reload(im)
failure_df, feature_names, target_columns, integer_encoding = im.preprocess_data(failure_data_path=failure_data_file,
                                                                weather_data_path=weather_data_file,
                                                                power_load_data_path=power_load_file,
                                                                feature_names=feature_names,
                                                                cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                initial_MC_state_filter='all'
                                                                )

In [8]:
failure_df

,Load,24h_min_load,Wind_speed,FDD,CDD3d,HDD3d,2d_load_sum,Extreme_cold,Holiday,Tmean,...,Season_sin,Season_cos,Month_sin,Month_cos,DayOfWeek_sin,DayOfWeek_cos,DayOfYear_sin,DayOfYear_cos,Final_gen_state,Data_weight
0,12349.0,12349.0,3.009058,0.000000,0.0,16.480059,24932.0,0.000000,0.0,2.147604,...,8.660254e-01,-0.5,-0.281733,-0.959493,0.866025,-0.5,0.025818,-0.999667,0.0,2.0
1,12349.0,12349.0,3.009058,0.000000,0.0,16.480059,24932.0,0.000000,0.0,2.147604,...,8.660254e-01,-0.5,-0.281733,-0.959493,0.866025,-0.5,0.025818,-0.999667,0.0,1.0
2,12349.0,12349.0,3.009058,0.000000,0.0,16.480059,24932.0,0.000000,0.0,2.147604,...,8.660254e-01,-0.5,-0.281733,-0.959493,0.866025,-0.5,0.025818,-0.999667,0.0,6.0
3,12349.0,12349.0,3.009058,0.000000,0.0,16.480059,24932.0,0.000000,0.0,2.147604,...,8.660254e-01,-0.5,-0.281733,-0.959493,0.866025,-0.5,0.025818,-0.999667,0.0,8.0
4,12349.0,12349.0,3.009058,0.000000,0.0,16.480059,24932.0,0.000000,0.0,2.147604,...,8.660254e-01,-0.5,-0.281733,-0.959493,0.866025,-0.5,0.025818,-0.999667,0.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44095152,94357.0,82737.0,3.896528,0.000000,0.0,17.273119,4156353.0,0.000000,1.0,0.636101,...,-2.449294e-16,1.0,0.000000,1.000000,0.866025,-0.5,0.000000,1.000000,2.0,2.0
44095153,94357.0,82737.0,3.896528,0.000000,0.0,17.273119,4156353.0,0.000000,1.0,0.636101,...,-2.449294e-16,1.0,0.000000,1.000000,0.866025,-0.5,0.000000,1.000000,2.0,1.0
44095154,77549.0,62996.0,3.700625,0.084727,0.0,18.173767,3370337.0,0.000000,1.0,-0.042792,...,-2.449294e-16,1.0,0.000000,1.000000,0.866025,-0.5,0.000000,1.000000,2.0,1.0
44095155,77549.0,62996.0,3.700625,0.084727,0.0,18.173767,3370337.0,0.000000,1.0,-0.042792,...,-2.449294e-16,1.0,0.000000,1.000000,0.866025,-0.5,0.000000,1.000000,2.0,4.0


In [18]:
failure_df.memory_usage(deep=True) / (1024 ** 2)  # in MB

Index                0.000126
FDD                336.419350
FDD3d              336.419350
Weekend            336.419350
Precipitation      336.419350
                      ...    
DayOfWeek_cos      336.419350
DayOfYear_sin      336.419350
DayOfYear_cos      336.419350
Final_gen_state    336.419350
Data_weight        336.419350
Length: 93, dtype: float64

In [19]:
failure_df.memory_usage(deep=True).sum() / (1024 ** 2)  # in MB

np.float64(30950.580295562744)

In [25]:
target_columns

['Final_gen_state']

In [4]:
importlib.reload(im)
def count_data(technologies):
    print(f"Counting data for technologies: {technologies}")
    states = ['A', 'D', 'U']
    counts = defaultdict(lambda: defaultdict(int))
    for si in states:
        print(f"  Initial state: {si}")
        init_df , _, _, integer_encoding = im.preprocess_data(failure_data_path=failure_data_file,
                                                                weather_data_path=weather_data_file,
                                                                power_load_data_path=power_load_file,
                                                                feature_names=feature_names,
                                                                cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                initial_MC_state_filter=si,
                                                                technology_filter=technologies
                                                                )
        for sf, sf_code in integer_encoding['Final_gen_state'].items():
            count = len(init_df[init_df['Final_gen_state'] == sf_code])
            counts[si][sf] = count
    return pd.DataFrame(counts)

In [22]:
integer_encoding['Technologies']

{'CC GT units ': 0,
 'CC steam units': 1,
 'Co-generator Block ': 2,
 'CoG GT units': 3,
 'CoG steam units ': 4,
 'Combined Cycle Block': 5,
 'Fluidized Bed': 6,
 'Fossil-Steam': 7,
 'Gas Turbine/Jet Engine (Simple Cycle Operation)': 8,
 'Gas Turbine/Jet Engine with HSRG': 9,
 'Geothermal': 10,
 'Internal Combustion/Reciprocating Engines': 11,
 'Miscellaneous': 12,
 'Multi-boiler/Multi-turbine': 13,
 'Nuclear': 14,
 'Pumped Storage/Hydro': 15}

In [5]:
nuc_tech = ['Nuclear']
hydro_tech = ['Pumped Storage/Hydro']
geothermal_tech = ['Geothermal']
thermal_tech = ['CC GT units ', 'CC steam units', 'Co-generator Block ', 'CoG GT units', 'CoG steam units ', 'Combined Cycle Block', 'Fluidized Bed', 'Fossil-Steam', 'Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Gas Turbine/Jet Engine with HSRG', 'Internal Combustion/Reciprocating Engines','Multi-boiler/Multi-turbine']

In [7]:
importlib.reload(im)
count_nuc = count_data(nuc_tech)
print(count_nuc)
print(f"    Total nuclear entries: {count_nuc.values.sum()}\n")
count_hydro = count_data(hydro_tech)
print(count_hydro)
print(f"    Total hydro entries: {count_hydro.values.sum()}\n")
count_geo = count_data(geothermal_tech)
print(count_geo)
print(f"    Total geothermal entries: {count_geo.values.sum()}\n")
count_thermal = count_data(thermal_tech)
print(count_thermal)
print(f"    Total thermal entries: {count_thermal.values.sum()}\n")

Counting data for technologies: ['Nuclear']
  Initial state: A
  Initial state: D
  Initial state: U
         A       D       U
A  2181184    3012     311
D     2928  633172     264
U      370     199  108113
    Total nuclear entries: 2929553

Counting data for technologies: ['Pumped Storage/Hydro']
  Initial state: A
  Initial state: D
  Initial state: U
         A       D        U
A  2935393    2216    23278
D     2163  823732      207
U    22651     170  1152211
    Total hydro entries: 4962021

Counting data for technologies: ['Geothermal']
  Initial state: A
  Initial state: D
  Initial state: U
        A      D      U
A  210507    670   1043
D     667  40421     35
U     897     33  37660
    Total geothermal entries: 291933

Counting data for technologies: ['CC GT units ', 'CC steam units', 'Co-generator Block ', 'CoG GT units', 'CoG steam units ', 'Combined Cycle Block', 'Fluidized Bed', 'Fossil-Steam', 'Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Gas Turbine/Jet Engine

# Training

In [4]:
nuc_tech = ['Nuclear']
hydro_tech = ['Pumped Storage/Hydro']
geothermal_tech = ['Geothermal']
thermal_tech = ['CC GT units ', 'CC steam units', 'Co-generator Block ', 'CoG GT units', 'CoG steam units ', 'Combined Cycle Block', 'Fluidized Bed', 'Fossil-Steam', 'Gas Turbine/Jet Engine (Simple Cycle Operation)', 'Gas Turbine/Jet Engine with HSRG', 'Internal Combustion/Reciprocating Engines','Multi-boiler/Multi-turbine']

technologies = thermal_tech

In [ ]:
initial_state = 'A'
test_periods = [(pd.Timestamp('2022-01-01'), pd.Timestamp('2023-12-31'))]

In [43]:
importlib.reload(im)
train_val_df, test_df, feature_names, target_columns, integer_encoding = im.preprocess_data(failure_data_path=failure_data_file,
                                                                                weather_data_path=weather_data_file,
                                                                                power_load_data_path=power_load_file,
                                                                                feature_names=feature_names,
                                                                                cyclic_features=["Season", "Month", "DayOfWeek", "DayOfYear"],
                                                                                initial_MC_state_filter=initial_state,
                                                                                technology_filter=technologies,
                                                                                test_periods=test_periods
                                                                                )

In [44]:
print(f"Train/Val Dataset shape: {train_val_df.shape}")
print(f"Test Dataset shape: {test_df.shape}")
print(f"Feature names ({len(feature_names)}): {feature_names}")
print(f"Target columns: {target_columns}")

Train/Val Dataset shape: (18444376, 86)
Test Dataset shape: (3583396, 86)
Feature names (84): ['State_MISSOURI', 'Wet_bulb_temperature', 'State_TEXAS', 'State_IDAHO', 'CDD3d', 'Dew_point_temperature', 'FDD', 'State_NORTH CAROLINA', 'State_VERMONT', 'State_WISCONSIN', 'Extreme_cold', 'FDD3d', 'State_MINNESOTA', 'State_MAINE', 'State_OHIO', 'State_GEORGIA', 'State_VIRGINIA', 'State_SOUTH DAKOTA', 'State_KENTUCKY', 'State_MICHIGAN', 'Snow_depth', 'State_CONNECTICUT', 'State_DELAWARE', 'State_FLORIDA', 'State_NEW YORK', 'State_LOUISIANA', 'State_NEW JERSEY', 'State_ARIZONA', 'Extreme_heat', 'Tmax', 'State_ILLINOIS', 'State_NEW HAMPSHIRE', 'Weekend', '24h_max_load', 'Precip_1d', 'State_RHODE ISLAND', 'Technology', 'Tmean', 'State_WYOMING', 'State_OREGON', '2d_load_sum', 'Station_level_pressure', 'Sea_level_pressure', 'State_TENNESSEE', 'Relative_humidity', 'State_MISSISSIPPI', 'State_ALABAMA', 'Precip_3d', 'Wind_chill_isnan', 'State_PENNSYLVANIA', 'HDD3d', 'Tmin', 'State_KANSAS', 'State_IOW

## XGBoost

In [ ]:
importlib.reload(im)

dataset_subset = dataset#.iloc[:1000, :].copy()

xgb_model = im.xgboostModel(verbose=True)
xgb_model.build_model(max_depth=6,
                      eta=0.07,
                      gamma=0.8,
                      reg_lambda=1,
                      num_boost_round=100,
                      feature_cols=feature_names,
                      target_cols=target_columns,
                      num_classes=3,
                      eval_metric='mlogloss', # rmse, logloss, mae, mape
                      objective='multi:softprob',
                      subsample=1)

xgb_model.prepare_data(dataset_subset, train_ratio=0.80, val_ratio=0.1, test_ratio=0.1, standardize=False)#, standardize=stand_cols_freq)

xgb_model.train_model(weights_data=True)

In [38]:
xgb_model.predict(dataset.iloc[:5, :])

array([[0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749],
       [0.98982936, 0.00523307, 0.00493749]], dtype=float32)

## MLP

In [8]:
importlib.reload(im)

dataset_subset = dataset.iloc[:1000, :].copy()

mlp = im.MLP(verbose=True)
mlp.build_model(feature_cols=feature_names, target_cols=target_columns, num_classes=3,
                hidden_sizes=[100, 100, 100, 50], activations=['relu', 'relu', 'relu', 'relu'])

mlp.prepare_data(dataset_subset, train_ratio=0.80, val_ratio=0.1)#, standardize=stand_cols_freq)


Sequential(
  (linear_0): Linear(in_features=92, out_features=100, bias=True)
  (activation_0): ReLU()
  (linear_1): Linear(in_features=100, out_features=100, bias=True)
  (activation_1): ReLU()
  (linear_2): Linear(in_features=100, out_features=100, bias=True)
  (activation_2): ReLU()
  (linear_3): Linear(in_features=100, out_features=50, bias=True)
  (activation_3): ReLU()
  (linear_out): Linear(in_features=50, out_features=3, bias=True)
)
Input dim: 92 | Output dim: 3 | Trainable params: 34,703


In [9]:
mlp.train_model(optimizer='adam', loss='cross_entropy',
                regularization_type='L2', lambda_reg=1e-3,
                weights_data=True,
                epochs=10, batch_size=10, lr=2e-4, device='cpu')

Starting MLP training...
Epoch 001: train=4.7319e+00 | val=3.8115e+00
Epoch 010: train=1.1208e+00 | val=5.0570e-01


# Debug

In [1]:
# Import libraries

# Data processing and manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.cluster import KMeans
import xgboost as xgb

# Other
import importlib
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))
import inferenceModels as im    # your file



# Mutual information
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif



from typing import Iterable
from pathlib import Path



importlib.reload(im)

<module 'inferenceModels' from '/Users/malo/MIT Dropbox/Malo Lahogue/Research/Generator-outage-probability/src/inferenceModels.py'>

In [2]:
weather_data_file = "../Data/hourly/hourly_weather_by_state.csv"
power_load_file = "../Data/hourly/hourly_load_by_state.csv"
failure_data_file = "../Data/hourly/hourly_failure_dataset_compressed.csv"

weather_data = pd.read_csv(weather_data_file, parse_dates=["datetime"])
power_load_data = pd.read_csv(power_load_file, parse_dates=["UTC time"])


feature_names = list(weather_data.columns) + list(power_load_data.columns) + ['Season', 'Month', 'DayOfWeek', 'DayOfYear', 'Holiday', 'Weekend', 'Technology']
feature_names = list(set(feature_names)-set(['datetime', 'UTC time', 'Datetime_UTC', 'Datetime']))
feature_names = list(set([(name[0].upper() + name[1:]) if isinstance(name, str) and name else name for name in feature_names]))
print(f"{len(feature_names)} features:", feature_names)

39 features: ['CDD3d', 'CDD', 'DayOfWeek', 'Heat_index', 'Wind_speed', 'Technology', '24h_max_load', '1d_load_sum', 'FDD3d', 'State', 'Dew_point_temperature', 'Load', 'Extreme_cold', 'Station_level_pressure', 'Tmin', 'Wind_chill', 'Temperature', 'Month', 'HDD', 'Precip_3d', 'Sea_level_pressure', 'Tmean', 'Wet_bulb_temperature', '24h_min_load', 'Pressure_3hr_change', 'Snow_depth', 'Season', '2d_load_sum', 'Tmax', 'Relative_humidity', 'Holiday', 'Precipitation', 'Weekend', 'DayOfYear', 'FDD', 'Hourly_load_change', 'HDD3d', 'Extreme_heat', 'Precip_1d']


In [3]:
dataset_subset = pd.read_csv("../Data/hourly/hourly_failure_dataset_thermal_A_subset.csv")
dataset_subset

,Relative_humidity,State_VIRGINIA,Extreme_cold,State_VERMONT,State_CALIFORNIA,Month_cos,State_OREGON,State_ALABAMA,HDD3d,State_MINNESOTA,...,State_DELAWARE,DayOfYear_cos,Dew_point_temperature,State_IDAHO,State_MASSACHUSETTS,Sea_level_pressure,State_NEW YORK,State_ILLINOIS,Final_gen_state,Data_weight
0,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,2.0
1,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,1.0
2,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,6.0
3,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,8.0
4,86.05637,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.480059,0.0,...,0.0,-0.999667,1.829314,0.0,0.0,1011.7077,0.0,0.0,0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,3.0
996,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,1.0
997,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,21.0
998,87.52599,0.0,0.0,0.0,0.0,-0.959493,0.0,0.0,16.291110,1.0,...,0.0,-0.999667,1.120612,0.0,0.0,1015.0276,0.0,0.0,0,36.0


In [4]:
target_columns = ['Final_gen_state']
feature_names = ['State_INDIANA', 'State_MISSISSIPPI', 'State_WASHINGTON', 'Pressure_3hr_change', 'Tmax', 'State_CONNECTICUT', 'State_WEST VIRGINIA', 'Season_sin', 'Station_level_pressure', 'Wind_speed', 'HDD3d', 'Dew_point_temperature', 'State_ILLINOIS', 'Season_cos', 'CDD3d', 'State_NEW HAMPSHIRE', 'Temperature', 'Technology', 'FDD3d', 'State_MAINE', 'State_GEORGIA', 'Holiday', 'DayOfYear_cos', 'Extreme_heat', 'DayOfYear_sin', 'Precip_3d', 'Sea_level_pressure', 'State_KANSAS', 'State_OREGON', 'State_MISSOURI', 'State_NEW YORK', 'Relative_humidity', 'State_SOUTH CAROLINA', 'State_OKLAHOMA', 'Hourly_load_change', '1d_load_sum', 'State_NEBRASKA', 'Wind_chill_isnan', 'State_DELAWARE', 'State_OHIO', 'Heat_index', 'State_TEXAS', 'State_ALABAMA', 'State_CALIFORNIA', 'Snow_depth', 'Month_sin', 'State_WYOMING', 'Precip_1d', 'FDD', '2d_load_sum', 'Wet_bulb_temperature', 'DayOfWeek_cos', 'HDD', 'Precipitation', 'State_MARYLAND', '24h_min_load', 'State_NORTH CAROLINA', 'State_ARKANSAS', 'Month_cos', 'State_SOUTH DAKOTA', 'State_MASSACHUSETTS', 'State_FLORIDA', 'State_COLORADO', 'State_UTAH', 'Tmean', 'State_NEVADA', 'Tmin', 'State_NORTH DAKOTA', 'State_LOUISIANA', 'State_TENNESSEE', 'State_MONTANA', 'State_PENNSYLVANIA', 'Load', 'State_VERMONT', 'Extreme_cold', 'Weekend', 'Heat_index_isnan', 'CDD', 'State_MINNESOTA', 'State_NEW JERSEY', 'State_KENTUCKY', 'DayOfWeek_sin', 'State_ARIZONA', '24h_max_load', 'State_IDAHO', 'Wind_chill', 'State_IOWA', 'State_VIRGINIA', 'State_MICHIGAN', 'State_RHODE ISLAND', 'State_NEW MEXICO', 'State_WISCONSIN']

In [5]:
print(f"Dataset shape: {dataset_subset.shape}")
print(f"Feature names ({len(feature_names)}): {feature_names}")
print(f"Target columns: {target_columns}")

Dataset shape: (1000, 94)
Feature names (92): ['State_INDIANA', 'State_MISSISSIPPI', 'State_WASHINGTON', 'Pressure_3hr_change', 'Tmax', 'State_CONNECTICUT', 'State_WEST VIRGINIA', 'Season_sin', 'Station_level_pressure', 'Wind_speed', 'HDD3d', 'Dew_point_temperature', 'State_ILLINOIS', 'Season_cos', 'CDD3d', 'State_NEW HAMPSHIRE', 'Temperature', 'Technology', 'FDD3d', 'State_MAINE', 'State_GEORGIA', 'Holiday', 'DayOfYear_cos', 'Extreme_heat', 'DayOfYear_sin', 'Precip_3d', 'Sea_level_pressure', 'State_KANSAS', 'State_OREGON', 'State_MISSOURI', 'State_NEW YORK', 'Relative_humidity', 'State_SOUTH CAROLINA', 'State_OKLAHOMA', 'Hourly_load_change', '1d_load_sum', 'State_NEBRASKA', 'Wind_chill_isnan', 'State_DELAWARE', 'State_OHIO', 'Heat_index', 'State_TEXAS', 'State_ALABAMA', 'State_CALIFORNIA', 'Snow_depth', 'Month_sin', 'State_WYOMING', 'Precip_1d', 'FDD', '2d_load_sum', 'Wet_bulb_temperature', 'DayOfWeek_cos', 'HDD', 'Precipitation', 'State_MARYLAND', '24h_min_load', 'State_NORTH CAROLIN

In [6]:
assert np.isfinite(dataset_subset.select_dtypes(float)).all().all()

In [7]:
importlib.reload(im)

mlp = im.MLP(verbose=True)
mlp.build_model(feature_cols=feature_names, target_cols=target_columns, num_classes=3,
                hidden_sizes=[100, 100, 100, 50], activations=['relu', 'relu', 'relu', 'relu'])

mlp.prepare_data(dataset_subset, train_ratio=0.80, val_ratio=0.1, test_ratio=0.1)#, standardize=stand_cols_freq)


Sequential(
  (linear_0): Linear(in_features=92, out_features=100, bias=True)
  (activation_0): ReLU()
  (linear_1): Linear(in_features=100, out_features=100, bias=True)
  (activation_1): ReLU()
  (linear_2): Linear(in_features=100, out_features=100, bias=True)
  (activation_2): ReLU()
  (linear_3): Linear(in_features=100, out_features=50, bias=True)
  (activation_3): ReLU()
  (linear_out): Linear(in_features=50, out_features=3, bias=True)
)
Input dim: 92 | Output dim: 3 | Trainable params: 34,703


In [11]:
mlp.train_model(optimizer='adam', loss='cross_entropy',
                regularization_type='L2', lambda_reg=1e-3,
                weights_data=True,
                epochs=100, batch_size=10, lr=2e-4, device='cpu')

Starting MLP training...
Epoch 001: train=6.2111e-01 | val=4.4007e+00
Epoch 010: train=3.1373e-01 | val=4.4397e+00
Epoch 020: train=9.7410e-02 | val=2.3857e-01
Epoch 030: train=1.0915e-01 | val=1.5469e+00
Epoch 040: train=5.7282e-02 | val=1.1337e+00
Epoch 050: train=1.1041e-01 | val=6.8859e-02
Epoch 060: train=5.5478e-02 | val=2.4298e-01
Epoch 070: train=2.3596e-02 | val=7.4670e-02
Epoch 080: train=3.2684e-02 | val=1.1864e-01
Epoch 090: train=2.6954e-02 | val=1.3077e-01
Epoch 100: train=1.8892e-02 | val=8.3244e-02
